In [ ]:
from pathlib import Path
import os
import sqlite3
import pandas as pd
from sklearn.datasets import load_breast_cancer

REPO_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = REPO_DIR / "data"
DATA_DIR.mkdir(exist_ok=True)

DB_PATH = DATA_DIR / "classification.db"
DB_PATH



WindowsPath('c:/Users/preon/OneDrive/Desktop/final project/housing_app_fall25/data/classification.db')

In [2]:
bc = load_breast_cancer(as_frame=True)
df = bc.frame.copy()  
df = df.rename(columns={"target": "label"})

#creating surrogate key here 
df = df.reset_index(drop=True)
df.insert(0, "row_id", df.index + 1)

df.shape, df["label"].value_counts(), df.head()


((569, 32),
 label
 1    357
 0    212
 Name: count, dtype: int64,
    row_id  mean radius  mean texture  mean perimeter  mean area  \
 0       1        17.99         10.38          122.80     1001.0   
 1       2        20.57         17.77          132.90     1326.0   
 2       3        19.69         21.25          130.00     1203.0   
 3       4        11.42         20.38           77.58      386.1   
 4       5        20.29         14.34          135.10     1297.0   
 
    mean smoothness  mean compactness  mean concavity  mean concave points  \
 0          0.11840           0.27760          0.3001              0.14710   
 1          0.08474           0.07864          0.0869              0.07017   
 2          0.10960           0.15990          0.1974              0.12790   
 3          0.14250           0.28390          0.2414              0.10520   
 4          0.10030           0.13280          0.1980              0.10430   
 
    mean symmetry  ...  worst texture  worst perimete

In [3]:
def build_3nf_sqlite_classification(db_path):
    print("=== BUILDING 3NF SQLITE DATA MODEL (CLASSIFICATION) ===")

    #split
    feature_cols = [c for c in df.columns if c not in ["row_id", "label"]]
    df_features = df[["row_id"] + feature_cols].copy()
    df_labels = df[["row_id", "label"]].copy()

    if os.path.exists(db_path):
        print("Existing DB found. Removing…")
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    #schema 
    cur.executescript("""
    DROP TABLE IF EXISTS labels;
    DROP TABLE IF EXISTS features;

    CREATE TABLE features (
        row_id INTEGER PRIMARY KEY
        -- feature columns will be added by pandas to_sql
    );

    CREATE TABLE labels (
        row_id INTEGER PRIMARY KEY,
        label INTEGER NOT NULL,
        FOREIGN KEY (row_id) REFERENCES features(row_id)
    );
    """)

    #data insert 
    df_features.to_sql("features", conn, if_exists="replace", index=False)
    df_labels.to_sql("labels", conn, if_exists="replace", index=False)

    cur.execute("CREATE INDEX IF NOT EXISTS idx_features_rowid ON features(row_id);")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_labels_rowid ON labels(row_id);")

    conn.commit()
    conn.close()

    print("=== DONE! SQLite DB created at:", db_path, "===\n")

build_3nf_sqlite_classification(str(DB_PATH))


=== BUILDING 3NF SQLITE DATA MODEL (CLASSIFICATION) ===
=== DONE! SQLite DB created at: c:\Users\preon\OneDrive\Desktop\final project\housing_app_fall25\data\classification.db ===



In [4]:
conn = sqlite3.connect(str(DB_PATH))

df_joined = pd.read_sql("""
SELECT f.*, l.label
FROM features f
JOIN labels l ON f.row_id = l.row_id
""", conn)

conn.close()

df_joined.shape, df_joined["label"].value_counts(), df_joined.head()

((569, 32),
 label
 1    357
 0    212
 Name: count, dtype: int64,
    row_id  mean radius  mean texture  mean perimeter  mean area  \
 0       1        17.99         10.38          122.80     1001.0   
 1       2        20.57         17.77          132.90     1326.0   
 2       3        19.69         21.25          130.00     1203.0   
 3       4        11.42         20.38           77.58      386.1   
 4       5        20.29         14.34          135.10     1297.0   
 
    mean smoothness  mean compactness  mean concavity  mean concave points  \
 0          0.11840           0.27760          0.3001              0.14710   
 1          0.08474           0.07864          0.0869              0.07017   
 2          0.10960           0.15990          0.1974              0.12790   
 3          0.14250           0.28390          0.2414              0.10520   
 4          0.10030           0.13280          0.1980              0.10430   
 
    mean symmetry  ...  worst texture  worst perimete